In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
import seaborn as sns
import pandas as pd

In [3]:
df = pd.read_csv('../data/raw/structure_projects.csv', sep=';', index_col=0)

In [4]:
df.head()

,floor_name,version,project_code,floor_code,software,slab_type,depth,floor_number,total_height,level,...,cantilever_mean_dist,cantilever_max_dist,cantilever_min_dist,cantilever_area,span_dists,cantilever_dists,center_x,center_y,slab_total_area,ignored
0,NIVEL +3.5,spans,BOCB19002_C00_I00_H00_Q00_R00_V00_S00_O01,BOCB19002_C00_I00_H00_Q00_R00_V00_S00_O01_F0,CYPE,FOAM_WAFFLE,0.3,4,13.00,3.5,...,3.904998,32.810214,0.000020,1473.747210,"[[9.900126260505125, 9.900126260505125, 9.9001...","[[0.11202188255563243, 0.11204643310773815, 0....",116.622403,20.353500,3292.093703,NO
1,NIVEL +7,spans,BOCB19002_C00_I00_H00_Q00_R00_V00_S00_O01,BOCB19002_C00_I00_H00_Q00_R00_V00_S00_O01_F1,CYPE,FOAM_WAFFLE,0.3,4,13.00,7.0,...,1.727113,10.700214,0.000000,307.297292,"[[7.499999999883002, 7.499999999883002, 7.4999...","[[0.37525137149771126, 0.911099628468607, 0.61...",108.302400,20.353500,881.073190,NO
2,NIVEL +10,spans,BOCB19002_C00_I00_H00_Q00_R00_V00_S00_O01,BOCB19002_C00_I00_H00_Q00_R00_V00_S00_O01_F2,CYPE,FOAM_WAFFLE,0.3,4,13.00,10.0,...,0.691147,8.290196,0.058214,135.735964,"[[9.900126260505125, 9.900126260505125, 9.9001...","[[0.10001375377489564, 0.14508624932332606, 0....",11.353856,21.003500,215.658464,NO
3,NIVEL +13,spans,BOCB19002_C00_I00_H00_Q00_R00_V00_S00_O01,BOCB19002_C00_I00_H00_Q00_R00_V00_S00_O01_F3,CYPE,FOAM_WAFFLE,0.3,4,13.00,13.0,...,0.161825,0.315655,0.100014,115.681666,"[[9.60013020613433, 9.60013020613433, 8.900140...","[[0.11202188255563521, 0.10003625895267909, 0....",11.353856,21.003500,230.876940,NO
0,Sótano 1,spans,BOCB19004_C00_I00_H00_Q00_R00_V00_S00_O01,BOCB19004_C00_I00_H00_Q00_R00_V00_S00_O01_F0,CYPE,FOAM_WAFFLE,0.3,18,58.45,-2.0,...,2.100032,6.436294,0.000032,134.408118,"[[23.843880609701003, 23.843880609701003, 9.87...","[[0.18106516001617137, 0.17977998363102696, 0....",25.619994,20.999437,803.681221,NO


In [5]:
num_total = len(df.groupby(by='project_code'))
ori_cond = df['project_code'].str.contains('_C00_I00_H00_Q00_R00_V00_S00_O01')
num_ori = len(df[ori_cond].groupby(by='project_code'))
opt_cond = df['project_code'].str.contains('_C00_I00_H00_Q00_R00_V00_S00_O02')
num_ori_opt = len(df[opt_cond].groupby(by='project_code'))
num_version = num_total - num_ori - num_ori_opt

print(f"Number of storeys: {len(df)}")
print(f"Number of projects: {num_total} {num_total}")
print(f"\tOriginal: {num_ori} {ori_cond.sum()} ({ori_cond.sum()/len(df)*100:.1f})")
print(f"\tOriginal optimized: {num_ori_opt} {opt_cond.sum()} ({opt_cond.sum()/len(df)*100:.1f})")
print(f"\tVersioned: {num_version}")


Number of storeys: 22636
Number of projects: 1927 1927
	Original: 250 2813 (12.4)
	Original optimized: 37 454 (2.0)
	Versioned: 1640


In [6]:
df.info(150)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22636 entries, 0 to 13
Data columns (total 105 columns):
 #    Column                             Dtype  
---   ------                             -----  
 0    floor_name                         object 
 1    version                            object 
 2    project_code                       object 
 3    floor_code                         object 
 4    software                           object 
 5    slab_type                          object 
 6    depth                              float64
 7    floor_number                       int64  
 8    total_height                       float64
 9    level                              float64
 10   height                             float64
 11   floor_max_height                   float64
 12   floor_min_height                   float64
 13   foundation_height                  float64
 14   max_height_all_floors              float64
 15   min_height_all_floors              float64
 16   suppo

In [7]:
s = pd.DataFrame()
s['feature_name'] = df.columns
s['type'] = [str(t) for t in df.dtypes.tolist()]
s['type'] = s['type'].replace({'object': 'string'})
s.set_index('feature_name', inplace=True)
s = s.join(df.describe().transpose())
cat_count = s[s['type'] == 'string']
s.loc[cat_count.index, 'count'] = df[cat_count.index.tolist()].notnull().sum()
s['unique_num'] = None
s.loc[cat_count.index, 'unique_num'] = pd.Series({col:len(df[col].unique()) for col in s[s['type'] == 'string'].index.tolist()})
s.loc[cat_count.index, 'unique_val'] = pd.Series({col:df[col].unique() for col in s[s['unique_num'] < 10].index.tolist()})
s

,type,count,mean,std,min,25%,50%,75%,max,unique_num,unique_val
feature_name,,,,,,,,,,,
floor_name,string,22636.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,919,NaN
version,string,22636.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,"[spans, 2.4.0]"
project_code,string,22636.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1927,NaN
floor_code,string,22636.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22636,NaN
software,string,22636.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,[CYPE]
...,...,...,...,...,...,...,...,...,...,...,...
cantilever_dists,string,22635.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5502,NaN
center_x,float64,22636.0,31.942705,58.128222,-2.087616,17.150589,24.062331,31.527242,2154.969696,None,NaN
center_y,float64,22636.0,18.113784,55.621427,-220.803849,14.850000,20.997200,28.181700,1789.047359,None,NaN


In [15]:
str_feat_df = s[s['type'] == 'string'][['type', 'count', 'unique_num', 'unique_val']].copy()
str_feat_df.to_latex('string_features.tex')
str_feat_df

/tmp/ipykernel_300831/3850106794.py:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  str_feat_df.to_latex('string_features.tex')


,type,count,unique_num,unique_val
feature_name,,,,
floor_name,string,22636.0,919,NaN
version,string,22636.0,2,"[spans, 2.4.0]"
project_code,string,22636.0,1927,NaN
floor_code,string,22636.0,22636,NaN
software,string,22636.0,1,[CYPE]
slab_type,string,22636.0,2,"[FOAM_WAFFLE, FLAT_SLAB]"
foundation_type,string,22636.0,3,"[FOOTING, PILES, GROUNDSLAB]"
period_x,string,22600.0,1923,NaN
period_y,string,22600.0,1923,NaN


In [16]:
num_feat_df = s[s['type'] != 'string'][['type', 'count', 'mean', 'std', 'min', 'max']]
num_feat_df.to_latex(
    'number_features.tex', bold_rows=True, float_format="%.2f")
num_feat_df

/tmp/ipykernel_300831/3516679795.py:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  num_feat_df.to_latex(


,type,count,mean,std,min,max
feature_name,,,,,,
depth,float64,22636.0,0.294173,0.037552,0.200000,0.400000
floor_number,int64,22636.0,15.147994,6.781881,2.000000,33.000000
total_height,float64,22636.0,43.168775,20.019927,0.850000,106.530000
level,float64,22636.0,22.036571,17.552202,-9.800000,106.530000
height,float64,22636.0,3.058745,0.496362,0.500000,7.000000
...,...,...,...,...,...,...
cantilever_min_dist,float64,22635.0,0.023716,0.063353,0.000000,2.395600
cantilever_area,float64,22635.0,160.091881,157.602925,-1311.025000,2389.402333
center_x,float64,22636.0,31.942705,58.128222,-2.087616,2154.969696


## Características con valores nulos

In [25]:
str_feat_df[str_feat_df['count'] < len(df)].index

Index(['period_x', 'period_y', 'span_dists', 'cantilever_dists', 'ignored'], dtype='object', name='feature_name')

In [27]:
num_feat_df[num_feat_df['count'] < len(df)].index

Index(['stick_model_displacement_vector_x',
       'stick_model_displacement_vector_y',
       'stick_model_displacement_vector_z', 'support_mean_span_dist',
       'support_max_span_dist', 'support_min_span_dist',
       'cantilever_mean_dist', 'cantilever_max_dist', 'cantilever_min_dist',
       'cantilever_area'],
      dtype='object', name='feature_name')